Treinamento completo 1! Tempo total: 0:46:13.768511

In [1]:
import os
import random
import torch
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from sklearn.model_selection import train_test_split

# Diretório das imagens processadas
output_dir = "/mnt/d/Documentos Gustavo/TCC/processed_heatmaps3"

# Listar IDs dos pacientes
healthy_patients = [p for p in os.listdir(os.path.join(output_dir, "Healthy")) if os.path.isdir(os.path.join(output_dir, "Healthy", p))]
sick_patients = [p for p in os.listdir(os.path.join(output_dir, "Sick")) if os.path.isdir(os.path.join(output_dir, "Sick", p))]

# Garantir aleatoriedade na separação
random.seed(42)
random.shuffle(healthy_patients)
random.shuffle(sick_patients)

# Descobrir o menor grupo
min_count = min(len(healthy_patients), len(sick_patients))
print('Por classe', min_count)
# Cortar ambos os grupos para o tamanho mínimo (balanceamento)
healthy_patients = healthy_patients[:min_count]
sick_patients = sick_patients[:min_count]

# Definir proporções (70% treino, 15% validação, 15% teste)
train_healthy, temp_healthy = train_test_split(healthy_patients, test_size=0.3, random_state=42)
val_healthy, test_healthy = train_test_split(temp_healthy, test_size=0.5, random_state=42)

train_sick, temp_sick = train_test_split(sick_patients, test_size=0.3, random_state=42)
val_sick, test_sick = train_test_split(temp_sick, test_size=0.5, random_state=42)

# Transformação das imagens
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

# Dataset personalizado para carregar 3 imagens por paciente
class CustomPatientDataset(Dataset):
    def __init__(self, patient_list, root_dir, transform=None):
        self.patient_list = patient_list
        self.root_dir = root_dir
        self.transform = transform
        self.classes = {"Healthy": 0, "Sick": 1}  # Rotulação

    def __len__(self):
        return len(self.patient_list)

    def __getitem__(self, idx):
        patient_id = self.patient_list[idx]
        
        # Determinar a classe do paciente (Healthy ou Sick)
        for category in ["Healthy", "Sick"]:
            patient_path = os.path.join(self.root_dir, category, patient_id)
            if os.path.exists(patient_path):
                label = self.classes[category]
                break

        # Listar imagens do paciente
        image_paths = [os.path.join(patient_path, img) for img in os.listdir(patient_path) if img.endswith(".jpg")]

        # Selecionar corretamente as imagens com base na 3ª parte da nomenclatura
        selected_images = {}
        for img in image_paths:
            partes = os.path.basename(img).split(".")
            if len(partes) >= 4:
                tipo = partes[2]  # A 3ª parte da nomenclatura indica o tipo da imagem
                if tipo in ["1", "3", "5"]:  # 1 = Frontal, 3 = Lateral esquerda, 5 = Lateral direita
                    selected_images[tipo] = img

        # Verificar se temos todas as imagens necessárias
        if not all(k in selected_images for k in ["1", "3", "5"]):
            raise ValueError(f"Paciente {patient_id} não possui todas as imagens necessárias!")

        # Carregar imagens e aplicar transformações
        images = [self.transform(Image.open(selected_images[k]).convert("RGB")) for k in ["1", "3", "5"]]

        # Concatenar para criar tensor de entrada (3 x C x H x W)
        images_tensor = torch.stack(images)

        return images_tensor, label

# Criar datasets de treino, validação e teste
train_dataset = CustomPatientDataset(train_healthy + train_sick, output_dir, transform)
val_dataset = CustomPatientDataset(val_healthy + val_sick, output_dir, transform)
test_dataset = CustomPatientDataset(test_healthy + test_sick, output_dir, transform)

# Criar DataLoaders
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

# Exibir quantidades finais
print(f"Tamanho do conjunto de treino: {len(train_dataset)} pacientes")
print(f"Tamanho do conjunto de validação: {len(val_dataset)} pacientes")
print(f"Tamanho do conjunto de teste: {len(test_dataset)} pacientes")


Por classe 120
Tamanho do conjunto de treino: 168 pacientes
Tamanho do conjunto de validação: 36 pacientes
Tamanho do conjunto de teste: 36 pacientes


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
import numpy as np
from datetime import datetime

# Configurar uso de GPU (se disponível)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Modificação do modelo DenseNet-121 para aceitar multiInput
class DenseNet121Modificada(nn.Module):
    def __init__(self):
        super(DenseNet121Modificada, self).__init__()
        self.num_subnets = 3
        self.num_features = 1024  # DenseNet-121 tem 1024 features na penúltima camada

        # Criar sub-redes (uma para cada imagem de entrada)
        self.subnets = nn.ModuleList([self._create_subnet() for _ in range(self.num_subnets)])

        # Camadas totalmente conectadas
        self.fc_final = nn.Sequential(
            nn.Linear(self.num_subnets * self.num_features, 4096),  # Primeira camada
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, 1024),  # Segunda camada
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1024, 2)  # Saída de 2 neurônios para classificação binária
        )

    def _create_subnet(self):
        base_model = models.densenet121(weights=models.DenseNet121_Weights.DEFAULT)
        num_features = base_model.classifier.in_features  # Obter número de features da DenseNet

        # Modificar a camada fully connected de cada subnet
        base_model.classifier = nn.Sequential(
            nn.Linear(num_features, 4096),  # Primeira camada
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, 1024),  # Segunda camada
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1024, 1024)  # Saída de 1024 neurônios para cada subnet
        )
        return base_model

    def forward(self, x):
        outputs = []
        for i in range(self.num_subnets):
            img = x[:, i, :, :, :]  # Selecionar a i-ésima imagem do batch
            outputs.append(self.subnets[i](img))  # Passar pela sub-rede correspondente
        
        concat = torch.cat(outputs, dim=1)  # Concatenar as saídas das sub-redes
        return self.fc_final(concat)  # Passar pela camada final


# Número de execuções e épocas
num_execucoes = 5
num_epochs = 30

# Listas para armazenar métricas de cada execução
accuracies = []
all_preds = []
all_probs = []

total_start_time = datetime.now()
for execucao in range(num_execucoes):
    print(f"\n🚀 Treinando Execução {execucao+1}/{num_execucoes}...\n")

    modelo = DenseNet121Modificada().to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = optim.Adam(modelo.parameters(), lr=0.0001, weight_decay=1e-4)

    # Early Stopping Configuração
    best_val_loss = float('inf')
    patience = 10
    epochs_no_improve = 0

    for epoch in range(num_epochs):
        modelo.train()
        train_loss = 0

        for imagens, labels in train_loader:
            imagens, labels = imagens.to(device), labels.to(device)

            # Não concatenamos as imagens! Apenas passamos normalmente para o modelo
            optimizer.zero_grad()
            output = modelo(imagens)
            loss = loss_fn(output, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        # Validação
        modelo.eval()
        val_loss = 0
        correct = 0
        total = 0
        with torch.no_grad():
            for imagens, labels in val_loader:
                imagens, labels = imagens.to(device), labels.to(device)

                output = modelo(imagens)
                loss = loss_fn(output, labels)
                val_loss += loss.item()

                _, predicted = torch.max(output, 1)
                correct += (predicted == labels).sum().item()
                total += labels.size(0)

        accuracy = 100 * correct / total
        avg_val_loss = val_loss / len(val_loader)

        print(f"Epoch {epoch+1}/{num_epochs} | Train Loss: {train_loss/len(train_loader):.4f} | "
              f"Val Loss: {avg_val_loss:.4f} | Val Acc: {accuracy:.2f}%")

        # Early Stopping
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(modelo.state_dict(), f"best_model_execucao_{execucao+1}.pth")
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= patience:
                print("🛑 Early Stopping ativado!")
                break

    # Avaliação no conjunto de teste
    modelo.load_state_dict(torch.load(f"best_model_execucao_{execucao+1}.pth"))
    modelo.eval()
    y_true = []
    y_pred = []
    y_prob = []

    with torch.no_grad():
        for imagens, labels in test_loader:
            imagens, labels = imagens.to(device), labels.to(device)

            output = modelo(imagens)
            probs = torch.softmax(output, dim=1)[:, 1]  # Probabilidades da classe 1 (positiva)
            _, preds = torch.max(output, 1)

            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy()) 
            y_prob.extend(probs.cpu().numpy()) 


    # Armazenar predições (probabilidades da classe positiva) para modelo médio
    all_preds.append(y_pred)
    all_probs.append(y_prob)

    # Armazenar acurácia binarizando as probabilidades
    preds_binarias = np.array(y_pred).round().astype(int)
    acc = (np.array(y_true) == preds_binarias).mean()
    accuracies.append(acc)
    print(f"✅ Execução {execucao+1} finalizada! Acurácia: {acc:.4f}")

    # 🚀 Liberação de memória após cada execução
    del modelo, optimizer, loss_fn
    torch.cuda.empty_cache()  # Libera memória da GPU

# Salvar predições e labels para análise posterior
np.save("all_preds.npy", all_preds)
np.save("all_probs.npy", all_probs)
np.save("y_true.npy", y_true)

total_end_time = datetime.now()
print(f"\n🎯 Treinamento completo! Tempo total: {total_end_time - total_start_time}")



🚀 Treinando Execução 1/5...

Epoch 1/30 | Train Loss: 0.6362 | Val Loss: 0.4754 | Val Acc: 80.56%
Epoch 2/30 | Train Loss: 0.3599 | Val Loss: 0.3225 | Val Acc: 80.56%
Epoch 3/30 | Train Loss: 0.4620 | Val Loss: 0.3025 | Val Acc: 83.33%
Epoch 4/30 | Train Loss: 0.0394 | Val Loss: 0.3706 | Val Acc: 88.89%
Epoch 5/30 | Train Loss: 0.0265 | Val Loss: 0.4072 | Val Acc: 94.44%
Epoch 6/30 | Train Loss: 0.1780 | Val Loss: 0.2625 | Val Acc: 91.67%
Epoch 7/30 | Train Loss: 0.2614 | Val Loss: 0.2376 | Val Acc: 91.67%
Epoch 8/30 | Train Loss: 0.1660 | Val Loss: 0.4023 | Val Acc: 83.33%
Epoch 9/30 | Train Loss: 0.0682 | Val Loss: 0.1668 | Val Acc: 97.22%
Epoch 10/30 | Train Loss: 0.0213 | Val Loss: 0.2387 | Val Acc: 94.44%
Epoch 11/30 | Train Loss: 0.0009 | Val Loss: 0.3547 | Val Acc: 91.67%
Epoch 12/30 | Train Loss: 0.3496 | Val Loss: 0.2719 | Val Acc: 94.44%
Epoch 13/30 | Train Loss: 0.2412 | Val Loss: 0.1859 | Val Acc: 91.67%
Epoch 14/30 | Train Loss: 0.1369 | Val Loss: 0.2654 | Val Acc: 86.11%

/tmp/ipykernel_67484/467332269.py:130: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  modelo.load_state_dict(torch.load(f"best_model_execucao_{execucao+1}.pth"))


✅ Execução 1 finalizada! Acurácia: 0.9444

🚀 Treinando Execução 2/5...

Epoch 1/30 | Train Loss: 0.6363 | Val Loss: 0.4357 | Val Acc: 80.56%
Epoch 2/30 | Train Loss: 0.4448 | Val Loss: 0.5212 | Val Acc: 80.56%
Epoch 3/30 | Train Loss: 0.1711 | Val Loss: 0.2325 | Val Acc: 88.89%
Epoch 4/30 | Train Loss: 0.1572 | Val Loss: 0.4838 | Val Acc: 86.11%
Epoch 5/30 | Train Loss: 0.2423 | Val Loss: 0.5694 | Val Acc: 88.89%
Epoch 6/30 | Train Loss: 0.1611 | Val Loss: 0.2949 | Val Acc: 91.67%
Epoch 7/30 | Train Loss: 0.2333 | Val Loss: 0.2271 | Val Acc: 91.67%
Epoch 8/30 | Train Loss: 0.0684 | Val Loss: 0.5136 | Val Acc: 88.89%
Epoch 9/30 | Train Loss: 0.0014 | Val Loss: 0.4858 | Val Acc: 88.89%
Epoch 10/30 | Train Loss: 0.0075 | Val Loss: 0.3543 | Val Acc: 94.44%
Epoch 11/30 | Train Loss: 0.0360 | Val Loss: 0.5479 | Val Acc: 91.67%
Epoch 12/30 | Train Loss: 0.0596 | Val Loss: 0.4988 | Val Acc: 94.44%
Epoch 13/30 | Train Loss: 0.0307 | Val Loss: 0.3342 | Val Acc: 94.44%
Epoch 14/30 | Train Loss: 0

# Modelo Médio POR VOTAÇÃO

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc

# Carregar predições salvas
all_preds = np.load("all_preds.npy")
y_true = np.load("y_true.npy")

# Calcular modelo médio por votação
final_preds = np.mean(np.array(all_preds), axis=0).round().astype(int)

# Gerar matriz de confusão
conf_matrix_final = confusion_matrix(y_true, final_preds)
tn, fp, fn, tp = conf_matrix_final.ravel()

# Calcular métricas do modelo médio
accuracy_final = accuracy_score(y_true, final_preds)
precision_final = precision_score(y_true, final_preds, average="binary")
recall_final = recall_score(y_true, final_preds, average="binary")
specificity_final = tn / (tn + fp)
f1_final = f1_score(y_true, final_preds, average="binary")

# Exibir métricas do modelo médio
print("\n===== 📊 MÉTRICAS DO MODELO MÉDIO =====")
print(f"✅ Accuracy: {accuracy_final:.4f}")
print(f"✅ Sensitivity (Recall): {recall_final:.4f}")
print(f"✅ Specificity: {specificity_final:.4f}")
print(f"✅ Precision: {precision_final:.4f}")
print(f"✅ F1-score: {f1_final:.4f}")

# Exibir matriz de confusão
plt.figure(figsize=(6, 5))
sns.heatmap(conf_matrix_final, annot=True, fmt="d", cmap="Blues",
            xticklabels=["Healthy", "Sick"],
            yticklabels=["Healthy", "Sick"])
plt.xlabel("Predição")
plt.ylabel("Real")
plt.title("Matriz de Confusão - Modelo Médio")
plt.show()

# Calcular curva ROC
all_probs = np.load("all_probs.npy")
fpr, tpr, _ = roc_curve(y_true, np.mean(np.array(all_probs), axis=0))
roc_auc = auc(fpr, tpr)

# #Experimento
# mean_probs = np.mean(np.array(all_probs), axis=0)
# print(np.unique(mean_probs))

# Exibir curva ROC
plt.figure(figsize=(6, 5))
plt.plot(fpr, tpr, color='blue', lw=2, label=f'Área sob a curva (AUC) = {roc_auc:.4f}')
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Taxa de Falsos Positivos')
plt.ylabel('Taxa de Verdadeiros Positivos')
plt.title('Curva ROC - Modelo Médio')
plt.legend(loc='lower right')
plt.show()




===== 📊 MÉTRICAS DO MODELO MÉDIO =====
✅ Accuracy: 0.9167
✅ Sensitivity (Recall): 0.9444
✅ Specificity: 0.8889
✅ Precision: 0.8947
✅ F1-score: 0.9189


[[1.64593197e-02 1.40208774e-03 4.15176433e-03 7.95764744e-01
  2.30747610e-01 2.90826917e-01 5.51142218e-03 5.46722829e-01
  1.18782875e-04 1.14937546e-04 4.55058903e-01 6.48764777e-04
  1.66322459e-02 2.00963550e-04 8.10566451e-03 1.46175560e-03
  8.10340280e-04 3.82410854e-01 9.82023656e-01 9.82023656e-01
  9.99999881e-01 9.99998808e-01 9.99997497e-01 9.99999642e-01
  9.95387137e-01 5.74028671e-01 9.99999881e-01 9.99999881e-01
  9.99998689e-01 5.43272495e-01 8.25750709e-01 9.99999881e-01
  9.99997616e-01 9.99999642e-01 9.99999762e-01 8.69509101e-01]
 [2.76442736e-01 1.42145813e-01 3.85344744e-01 9.92239356e-01
  9.31779444e-01 4.15825933e-01 3.45890433e-01 1.41760841e-01
  1.92430228e-01 1.03451513e-01 5.59962392e-01 2.13797569e-01
  4.04654413e-01 1.17884584e-01 1.62265152e-01 2.97974765e-01
  2.98919111e-01 1.89488128e-01 9.90023553e-01 9.90023553e-01
  9.99999881e-01 9.99999881e-01 1.00000000e+00 9.99999881e-01
  9.93605614e-01 9.90079165e-01 1.00000000e+00 1.00000000e+00
  1.000